In [ ]:
 
import spacy  
from scipy.spatial.distance import cosine
import pandas as pd
import numpy as np
  
import matplotlib.pyplot as plt  
import matplotlib.cm as cm  
import seaborn as sns  
    
from sklearn.decomposition import PCA  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

#Ignora errores por el tamaño del csv que no permite hacer todas las iteraciones necesarias
import warnings
warnings.filterwarnings("ignore")

<center><img src="https://i.ibb.co/zR397jW/Tarea5-portada.png" alt="Portada tarea 5"></center>

In [2]:
## Datos de csv, se demora a lo mucho:  16 seg por el tamaño del csv
parlamentarios = pd.read_csv('parlamentarios_info_general.csv', delimiter=';')
participacion = pd.read_csv('participacion_descripcion.csv')

### Limpieza de datos

In [3]:
# Eliminaré todos los valores innecesarios
parla_minus = ['VIGENTE' , 'FECHA_NACIMIENTO' , 'SITIOS_WEB' , 'CAMARA_ACTUAL' , 'CAMARA' , 'INICIO' , 'FINAL' , 'UNIDAD_QUE_REPRESENTA']
parti_minus = ['ID_PARTICIPACION' , 'TIPO_SESION' , 'FECHA', 'TIPO_PARTICIPACION' , 'CAMARA' , 'PARLAMENTARIOS', 'BOLETIN_ID', 'PARLAMENTARIOS', 'BOLETIN_ID', 'LEGISLATURA', 'SESION', 'TEXTO_ANTECEDENTE', 'DESCRIPCION_DEBATE']

In [4]:
# Definimos los partidos
partidos_derecha = [
    "Partido Unión Demócrata Independiente",
    "Partido Republicano de Chile",
    "Partido Renovación Nacional",
    "Partido Social Cristiano",
    "Partido Evolución Política (Evópoli)",
    "Partido Demócratas Chile",
    "Movimiento Amarillos por Chile"
]

partidos_izquierda = [
    "Partido Comunista de Chile",
    "Partido Por la Democracia",
    "Partido Frente Amplio",
    "Partido Socialista de Chile",
    "Partido Radical Socialdemócrata",
    "Partido Demócrata Cristiano",
    "Partido Liberal de Chile",
    "Federación Regionalista Verde Social",
    "Partido Ecologista Verde",
    "Partido Humanista",
    "Partido Acción Humanista"
]

In [5]:
party_mapping = {}

# Partidos de izquierda serán 0
for party in partidos_izquierda:
    party_mapping[party] = 0

# Partidos de derecha serán 1
for party in partidos_derecha:
    party_mapping[party] = 1

# Si no es ninguno será 2
party_mapping['default'] = 2

In [6]:
parlamentarios['PARTIDO_POLITICO'] = parlamentarios['PARTIDO_POLITICO'].map(party_mapping).fillna(2).astype(int)

In [7]:
parlamentarios.drop(columns=parla_minus, inplace=True, errors='ignore')
parlamentarios['GENERO'] = parlamentarios['GENERO'].replace({'M': 0, 'F': 1}).infer_objects()
parlamentarios['GENERO'] = parlamentarios['GENERO'].astype(int)
parlamentarios.head(10)


,PARLAMENTARIO_ID,NOMBRE_COMPLETO,GENERO,PARTIDO_POLITICO
0,4611,Cristhian Moreira Barros,0,1
1,5209,Daniela Serrano Salazar,1,0
2,5194,Cristian Tapia Ramos,0,2
3,5203,Cristian Araya Lerdo De Tejada,0,1
4,4647,Cristian Labbe Martinez,0,2
5,5265,Cristobal Martinez Ramirez,0,1
6,4549,Cristobal Urruticoechea Rios,0,2
7,4589,Luz Eliana Ebensperger Orrego,1,1
8,4629,Raul Soto Mardones,0,0
9,4576,Maite Orsini Pascal,1,0


In [8]:
participacion.drop(columns=parti_minus, inplace=True, errors='ignore')
participacion.head(10)

,PARLAMENTARIO_ID,TEXTO_PRINCIPAL
0,3494,"La señora ALLENDE.- \nGracias, Presidente. \nH..."
1,3494,"La señora ALLENDE.- \nGracias, Presidente. \nC..."
2,3494,"La señora ALLENDE.- \nGracias, Presidente. \nP..."
3,3494,"La señora ALLENDE.- \nPresidente , yo le pedí ..."
4,3494,"La señora ALLENDE.- \nPresidente, como ha seña..."
5,3494,Proyecto de Acuerdo de los Honorables Senadore...
6,3494,"La señora ALLENDE.- \nGracias, Presidente. \nH..."
7,3494,-Proyecto de acuerdo de los Honorables Senador...
8,3494,"La señora ALLENDE.- \nGracias, Presidente. \nC..."
9,3494,-Proyecto de acuerdo de los Honorables Senador...


In [9]:
# Ahora lo uniremos para poder hacer las predicciones
final = pd.merge(participacion, parlamentarios[['PARLAMENTARIO_ID', 'GENERO', 'PARTIDO_POLITICO']], on='PARLAMENTARIO_ID', how='left')
final['TEXTO_PRINCIPAL'] = final['TEXTO_PRINCIPAL'].fillna('')
final


,PARLAMENTARIO_ID,TEXTO_PRINCIPAL,GENERO,PARTIDO_POLITICO
0,3494,"La señora ALLENDE.- \nGracias, Presidente. \nH...",1,0
1,3494,"La señora ALLENDE.- \nGracias, Presidente. \nC...",1,0
2,3494,"La señora ALLENDE.- \nGracias, Presidente. \nP...",1,0
3,3494,"La señora ALLENDE.- \nPresidente , yo le pedí ...",1,0
4,3494,"La señora ALLENDE.- \nPresidente, como ha seña...",1,0
...,...,...,...,...
245745,4622,"-Asistieron 151 señores diputados, de los 155 ...",1,0
245746,4622,"-Asistieron 154 señores diputados, de los 155 ...",1,0
245747,4622,El señor MULET (Vicepresidente).- \n \n Tiene ...,1,0
245748,4622,"-Asistieron 151 señores diputados, de los 155 ...",1,0


# Parte 1. model_gen de género 
Debes implementar un model_gen que reciba un string y determine si
ese string corresponde a un discurso hecho por un hombre o una mujer. Para esto puedes usar cualquiera de
los métodos y algoritmos vistos en clases, pero no puedes usar algo que no hayamos visto. Tu model_gen
debe ser implementado mediante una clase ClasText con dós metodos:
Un método fit, que recibe una lista o un array de strings X, y una lista, array o series y que solo
contiene números 0 o 1. Este método entrena un model_gen que puede predecir si un string dado debe
ser etiquetado como 0 o 1, en base a lo visto en el entrenamiento.
Un método predict, que recibe una lista o un array de strings X, y entrega una lista o un array de
números 0 o 1 del mismo tamaño que X.


In [10]:
# Creamos ClasText
class ClasText:
    def __init__(self):
        self.model = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=245749))
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict(self, X):
        return self.model.predict(X)

In [11]:
model_gen = ClasText()

In [12]:
# Ahora X_gen será la nueva columna de texto combinado. demora a lo mucho:  en correr 1 minuto y 15 seg.

X_gen = final['TEXTO_PRINCIPAL']
y = final['GENERO']

X_train_genero, X_test_gen, y_train_gen, y_test_gen = train_test_split(X_gen, y, test_size=0.3, random_state=42)

model_gen.fit(X_train_genero , y_train_gen)

In [13]:
# La celda demora a lo mucho:  30s
y_pred = model_gen.predict(X_test_gen)

# Parte 2. Análisis 
Contesta usando las herramientas vistas en clases: ¿Cómo es el rendimiento de tu
model_gen? Prueba ahora otros dos clasificadores distintos, usando otros algoritmos u otras formas de
procesar texto. ¿Mejora el rendimiento?

In [14]:
class_report = classification_report(y_test_gen, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     53428
           1       0.83      0.66      0.74     20297

    accuracy                           0.87     73725
   macro avg       0.86      0.81      0.83     73725
weighted avg       0.87      0.87      0.87     73725



Se comporta de una manera bastante buena. Tiene una precisión y recall alta. Es mejor para predecir hombres que mujeres. 

### Comparación 1: MultinomialNB y TfidVectorizer

El [MultinomalNB](https://www.geeksforgeeks.org/multinomial-naive-bayes/) porque...
Mantendré TfidVectorizaer para transformar texto a vectores. 
Este nuevo modelo busca ver si es que el culpable de las inexactitudes es el ```LogisticRegression```

In [15]:
## Usaré MultinomialNB. Retirado de: # 
## Nuevamente para traducir, usaré TfidVectorizer
class Compare1:
    def __init__(self):
        self.model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict(self, X):
        return self.model.predict(X)

In [16]:
# La celda se demora a lo mucho:  1 min y 10 seg
clasificador_2 = Compare1()

clasificador_2.fit(X_train_genero, y_train_gen)

In [17]:
y_pred_2 = clasificador_2.predict(X_test_gen)
class_report = classification_report(y_test_gen, y_pred_2)
print(class_report)

              precision    recall  f1-score   support

           0       0.74      0.99      0.85     53428
           1       0.79      0.09      0.16     20297

    accuracy                           0.74     73725
   macro avg       0.76      0.54      0.50     73725
weighted avg       0.75      0.74      0.66     73725



Vemos que al cambiar de ```LogisticRegression``` a ```MultinomialNB``` empeora la calidad del model_gen. Así que no es el culpable de las inexactitudes. 

### Comparación 1: LogisticRegression y Count Vectorizer

El [CountVectorizer](https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/) porque...
Mantendré LogisticRegression para transformar texto a vectores. 
Este nuevo modelo busca ver si es que el culpable de las inexactitudes es el ```TfidVectorizaer```

In [18]:
## Usaré Count Vectorizer 
## Nuevamente para predecir usaré LogisticRegression
class Compare2:
    def __init__(self):
        self.model = make_pipeline(CountVectorizer(), LogisticRegression())
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict(self, X):
        return self.model.predict(X)

In [19]:
## Se demora a lo mucho:  1 minuto y 15 seg

clasificador_3 = Compare2()

clasificador_3.fit(X_train_genero, y_train_gen)

In [ ]:
# Se demora a lo mucho: 25 seg

y_pred = clasificador_3.predict(X_test_gen)
class_report = classification_report(y_test_gen, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.83      0.94      0.88     53428
           1       0.76      0.48      0.59     20297

    accuracy                           0.81     73725
   macro avg       0.79      0.71      0.73     73725
weighted avg       0.81      0.81      0.80     73725



Similarmente, el hecho de cambiar el codificador a no mejora la performance del modelo. Parece que la mejor combinación posible es codificar con ```TfidVectorizaer``` y usar para predecir ```LogisticRegression```

# Parte 3. Extension: clasificando ideologıa polıtica 
Considera que los partidos de derecha son: Partido
Union Democrata Independiente, Partido Republicano de Chile, Partido Renovacion Nacional, Partido Social
Cristiano, Partido Evolución Polıtica (Evopoli), Partido Dem ́ocratas Chile y Movimiento Amarillos por Chile.

Considera que los partidos de izquierda son: Partido Comunista de Chile, Partido Por la Democracia,
Partido Frente Amplio, Partido Socialista de Chile, Partido Radical Socialdemócrata, Partido Demócrata
Cristiano, Partido Liberal de Chile, Federacion Regionalista Verde Social, Partido Ecologista Verde, Partido
Humanista y Partido Acción Humanista.

Muestra como puedes usar ClasText para clasificar discursos de alguien de izquierda o de derecha.

In [21]:
## Nuevamente definiré un modelo usando ClasText

model_party = ClasText()

X_party = final['TEXTO_PRINCIPAL']
y_party = final['PARTIDO_POLITICO']

X_train_party, X_test_party, y_train_party, y_test_party = train_test_split(X_party, y_party, test_size=0.3, random_state=42)


In [22]:
# La celda se demora a lo mucho:  2 minuto y 10 segundos
model_party.fit(X_train_party, y_train_party)

In [23]:
# La celda se demora a lo mucho:  15 seg

y_party_predict = model_party.predict(X_test_party)
y_party_predict

array([2, 0, 1, ..., 1, 1, 0])

In [24]:
unique_values = np.unique(y_party_predict)
unique_values

array([0, 1, 2])

Vemos que efectivamente predice que serán de derecha, izquierda u otros

In [25]:
class_report = classification_report(y_test_party, y_party_predict)
print(class_report)

              precision    recall  f1-score   support

           0       0.78      0.81      0.80     26840
           1       0.77      0.83      0.80     28983
           2       0.77      0.63      0.69     17902

    accuracy                           0.77     73725
   macro avg       0.77      0.76      0.76     73725
weighted avg       0.77      0.77      0.77     73725



Vemos que de manera similar a la parte 2, puede predecir el partido politico pero no es tan efectivo. 

# Parte 4. ¡Entregale nuevo texto a tus clasificadores! 
Ahora que tienes clasificadores, aprovecha de jugar un rato. Entregale textos que tu creas que pertenecen claramente a una clase u otra, y ve que predice tu model_gen. ¿Te hace sentido?

## Desarrollo
Decidí hacer una predicción basado en el projecto que hizo ***KamadoTanjiro-beep*** llamado [Book-Genre-Prediction](https://github.com/KamadoTanjiro-beep/Book-Genre-Prediction) disponible en GitHub. Usó 3000 sinopsis de libros para predecir el genero de cada uno. Voy a ver si puedo usarlo para predecir el genero del libro efectivamente. 

In [26]:
BookData = pd.read_csv("https://raw.githubusercontent.com/KamadoTanjiro-beep/Book-Genre-Prediction/refs/heads/master/BooksDataSet.csv")
# Quito columnas innecesarias.  
BookData = BookData.drop(columns=['book_id', 'Unnamed: 0', 'book_name'])

BookData

,genre,summary
0,Fantasy,Drowned Wednesday is the first Trustee among ...
1,Fantasy,"As the book opens, Jason awakens on a school ..."
2,Fantasy,Cugel is easily persuaded by the merchant Fia...
3,Fantasy,The book opens with Herald-Mage Vanyel return...
4,Fantasy,Taran and Gurgi have returned to Caer Dallben...
...,...,...
2995,Thriller,"A Novel from the NUMA files, A Kurt Austin Ad..."
2996,Thriller,Gilbert Kemp is dealer specializing in antiqu...
2997,Thriller,"""How do you know when you're in too deep? Dav..."
2998,Thriller,The story concerns the life of Johnnie Pascoe...


In [27]:
print(BookData['genre'].unique())

['Fantasy' 'Science Fiction' 'Crime Fiction' 'Historical novel' 'Horror'
 'Thriller']


Hay 6 generos considerados: Fantasia, Ciencia Ficción, Crimienes, Novelas historicas, Horror y Thriller. Los transformaré a valores númericos. 

In [28]:
## Fantasia es 0, Ciencia Ficción es 1, Crimienes es 2, Novelas historicas es 3, Horror es 4 y Thriller es 5
genre_mapping = {'Fantasy': 0, 'Science Fiction': 1, 'Crime Fiction': 2, 'Historical novel': 3, 'Horror': 4, 'Thriller': 5}

BookData['genre'] = BookData['genre'].map(genre_mapping)
BookData

,genre,summary
0,0,Drowned Wednesday is the first Trustee among ...
1,0,"As the book opens, Jason awakens on a school ..."
2,0,Cugel is easily persuaded by the merchant Fia...
3,0,The book opens with Herald-Mage Vanyel return...
4,0,Taran and Gurgi have returned to Caer Dallben...
...,...,...
2995,5,"A Novel from the NUMA files, A Kurt Austin Ad..."
2996,5,Gilbert Kemp is dealer specializing in antiqu...
2997,5,"""How do you know when you're in too deep? Dav..."
2998,5,The story concerns the life of Johnnie Pascoe...


In [29]:
## Definimos el modelo usando ClasText

book_model = ClasText()

X_book = BookData['summary']
y_book = BookData['genre']

X_train_book, X_test_book, y_train_book, y_test_book = train_test_split(X_book, y_book, test_size=0.2, random_state=42)

In [30]:
book_model.fit(X_train_book, y_train_book)

In [31]:
y_predicted_book = book_model.predict(X_test_book)

In [32]:
class_report = classification_report(y_test_book, y_predicted_book)
print(class_report)

              precision    recall  f1-score   support

           0       0.78      0.65      0.71       118
           1       0.69      0.75      0.71        99
           2       0.71      0.82      0.76        96
           3       0.65      0.83      0.73       101
           4       0.69      0.52      0.59        91
           5       0.63      0.56      0.59        95

    accuracy                           0.69       600
   macro avg       0.69      0.69      0.68       600
weighted avg       0.69      0.69      0.69       600



Vemos que es capas de predecir pero no tiene un accuracy muy alto  tampoco una gran precisión. Veamos como actua con ejemplos de libros famosos. 

Las sinopsis son de: 

Fantasia: [The Fellowship of the Ring de J.R.R. Tolkien](https://www.goodreads.com/book/show/3263607-the-fellowship-of-the-ring)

Ciencia Ficción: [The War of the Worlds de H.G. Wells](https://www.goodreads.com/book/show/8909.The_War_of_the_Worlds)

Crimienes: [The Girl with the Dragon Tattoo de Stieg Larsson](https://www.goodreads.com/book/show/2429135.The_Girl_with_the_Dragon_Tattoo)

Novelas historicas: [Gone with the Wind de Margaret Mitchell](https://www.goodreads.com/book/show/18405.Gone_with_the_Wind)

Horror: [Dracula de Bram Stoker](https://www.goodreads.com/book/show/17245.Dracula)

Thriller: [The Patient's Secret de Loreth Anne White](https://www.goodreads.com/book/show/57694667-the-patient-s-secret)


In [33]:
sinopsis = {
    0:"One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the darkness bind them. In ancient times the Rings of Power were crafted by the Elven-smiths, and Sauron, the Dark Lord, forged the One Ring, filling it with his own power so that he could rule all others. But the One Ring was taken from him, and though he sought it throughout Middle-earth, it remained lost to him. After many ages it fell into the hands of Bilbo Baggins, as told in The Hobbit. In a sleepy village in the Shire, young Frodo Baggins finds himself faced with an immense task, as his elderly cousin Bilbo entrusts the Ring to his care. Frodo must leave his home and make a perilous journey across Middle-earth to the Cracks of Doom, there to destroy the Ring and foil the Dark Lord in his evil purpose.",
    1:"When an army of invading Martians lands in England, panic and terror seize the population. As the aliens traverse the country in huge three-legged machines, incinerating all in their path with a heat ray and spreading noxious toxic gases, the people of the Earth must come to terms with the prospect of the end of human civilization and the beginning of Martian rule.",
    2:"Harriet Vanger, a scion of one of Sweden’s wealthiest families disappeared over forty years ago. All these years later, her aged uncle continues to seek the truth. He hires Mikael Blomkvist, a crusading journalist recently trapped by a libel conviction, to investigate. He is aided by the pierced and tattooed punk prodigy Lisbeth Salander. Together they tap into a vein of unfathomable iniquity and astonishing corruption.",
    3:"When Jonathan Harker visits Transylvania to help Count Dracula with the purchase of a London house, he makes a series of horrific discoveries about his client. Soon afterwards, various bizarre incidents unfold in England: an apparently unmanned ship is wrecked off the coast of Whitby; a young woman discovers strange puncture marks on her neck; and the inmate of a lunatic asylum raves about the 'Master' and his imminent arrival.", 
    4:"Secrets worth dying―and killing―for, in a novel of suspense inspired by a horrific true crime by Washington Post and Amazon Charts bestselling author Loreth Anne White. Lily Bradley is a respected psychotherapist married to a distinguished professor. They live in a dream house with their two children in close-knit Story Cove. Lily lives a well-ordered life. Or so it seems. As a therapist, she knows everyone keeps things hidden. Even her. Then sensual and free-spirited Arwen Harper rolls into town in her hand-painted VW van, her sixteen-year-old son riding shotgun. Overnight, Story Cove’s secrets are no longer safe. Because Arwen might know her new neighbors better than they know themselves. Now someone is dead, and it looks like murder. Brutal and personal. The death invites the shrewd eye of Detective Rue Duval. Rue’s job is to expose secrets. But she’s also an expert at keeping them.",
}

generos = {
    0: "Fantasia",
    1: 'Ciencia Ficción',
    2: 'Crimenes',
    3: 'Novelas Historicas',
    4: 'Horror',
    5: 'Thriller'
}


In [34]:

for key, synopsis in sinopsis.items():
    prediction = book_model.predict([synopsis])  # Wrap synopsis in a list for the model
    print(f"Genero Real: {generos[key]} --> Prediccion: {generos[prediction[0]]}")

Genero Real: Fantasia --> Prediccion: Fantasia
Genero Real: Ciencia Ficción --> Prediccion: Ciencia Ficción
Genero Real: Crimenes --> Prediccion: Crimenes
Genero Real: Novelas Historicas --> Prediccion: Horror
Genero Real: Horror --> Prediccion: Crimenes


Vemos que 3 de los 5 generos fueron adivinados. Así que el modelo tal vez con más entrenamiento será capaz de adivinar que genero mejor. Hay que considerar que hay temas comunes entre los crimenes y el horror. También que algunos libros de horror pasan en epocas historicas (Como Drácula). 